In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', -1)

## read, info, head

In [3]:
usecols=['id','title','genres','keywords','overview','popularity','production_companies','release_date','release_month',
         'tagline','vote_average','vote_count','cast_name','producer','director']

movie_df = pd.read_csv('./data/clean_movie_plus_credit.csv', encoding='raw_unicode_escape', usecols=usecols)

In [4]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 15 columns):
genres                  4775 non-null object
id                      4803 non-null int64
keywords                4390 non-null object
overview                4803 non-null object
popularity              4803 non-null float64
production_companies    4451 non-null object
release_date            4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
release_month           4803 non-null int64
cast_name               4757 non-null object
producer                3743 non-null object
director                4562 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 562.9+ KB


In [5]:
movie_df.head()

,genres,id,keywords,overview,popularity,production_companies,release_date,tagline,title,vote_average,vote_count,release_month,cast_name,producer,director
0,"Action,Adventure,Fantasy,Science Fiction",19995,"culture clash,future,space war,space colony,society,space travel,futuristic,romance,space,alien,tribe,alien planet,cgi,marine,soldier,battle,love affair,anti war,power relations,mind and soul,3d","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,"Ingenious Film Partners,Twentieth Century Fox Film Corporation,Dune Entertainment,Lightstorm Entertainment",2009-12-10,Enter the World of Pandora.,Avatar,7.2,11800,12,"Sam Worthington,Zoe Saldana,Sigourney Weaver,Stephen Lang,Michelle Rodriguez,Giovanni Ribisi,Joel David Moore,CCH Pounder,Wes Studi,Laz Alonso,Dileep Rao,Matt Gerald,Sean Anthony Moran,Jason Whyte,Scott Lawrence,Kelly Kilgour,James Patrick Pitt,Sean Patrick Murphy,Peter Dillon,Kevin Dorman,Kelson Henderson,David Van Horn,Jacob Tomuri,Jon Curry,Luke Hawker,Woody Schultz,Peter Mensah,Sonia Yee,Jahnel Curfman,Ilram Choi,Kyla Warren,Lisa Roumain,Debra Wilson,Chris Mala,Taylor Kibby,Jodie Landau,Julie Lamm,Cullen B. Madden,Joseph Brady Madden,Frankie Torres,Austin Wilson,Sara Wilson,Lucy Briant,Nathan Meister,Gerry Blair,Matthew Chamberlain,Paul Yates,Wray Wilson,James Gaylyn,Melvin Leno Clark III,Carvon Futrell,Brandon Jelkes,Micah Moch,Hanniyah Muhammad,Christopher Nolen,Christa Oliver,April Marie Thomas,Bravita A. Threatt,Colin Bleasdale,Mike Bodnar,Matt Clayton,Nicole Dionne,Jamie Harrison,Allan Henry,Anthony Ingruber,Ashley Jeffery,Dean Knowsley,Terry Notary,Kai Pantano,Logan Pithyou,Stuart Pollock,Raja,Gareth Ruck,Rhian Sheehan,T. J. Storm,Jodie Taylor,Richard Whiteside,Nikie Zambo,Julene Renee","James Cameron,Jon Landau",James Cameron
1,"Adventure,Fantasy,Action",285,"ocean,drug abuse,exotic island,east india trading company,love of one's life,traitor,shipwreck,strong woman,ship,alliance,calypso,afterlife,fighter,pirate,swashbuckler,aftercreditsstinger","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082615,"Walt Disney Pictures,Jerry Bruckheimer Films,Second Mate Productions",2007-05-19,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,5,"Johnny Depp,Orlando Bloom,Keira Knightley,Bill Nighy,Geoffrey Rush,Jack Davenport,Kevin McNally,Tom Hollander,Naomie Harris,Jonathan Pryce,Keith Richards,Lee Arenberg,Mackenzie Crook,Greg Ellis,David Bailie,Martin Klebba,David Schofield,Lauren Maher,Vanessa Branch,Angus Barnett,Giles New,Reggie Lee,Dominic Scott Kay,Takayo Fischer,David Meunier,Andy Beckwith,Peter Donald Badalamenti II,Christopher S. Capp,Keith Richards,Ghassan Massoud","Jerry Bruckheimer,Eric McLeod,Chad Oman,Peter Kohn,Pat Sandston",Gore Verbinski
2,"Action,Adventure,Crime",206647,"spy,based on novel,secret agent,sequel,mi6,british secret service,united kingdom","A cryptic message from BondÃÂ¢ÃÂÃÂs past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.",107.376788,"Columbia Pictures,Danjaq,B24",2015-10-26,A Plan No One Escapes,Spectre,6.3,4466,10,"Daniel Craig,Christoph Waltz,Ralph Fiennes,Monica Bellucci,Ben Whishaw,Naomie Harris,Dave Bautista,Andrew Scott,Rory Kinnear,Jesper Christensen,Alessandro Cremona,Stephanie Sigman,Tenoch Huerta,Adriana Paz,Domenico Fortunato,Marco Zingaro,Stefano Elfi DiClaudia,Ian Bonar,Tam Williams,Richard Banham,Pip Carter,Simon Lenagan,Alessandro Bressanello,Marc Zinga,Brigitte Millar,Adel Bencherif,Gediminas Adomaitis,Peppe Lanzetta,Francesco Arca,Matteo Taranto,Emilio Aniba,Benito Sagredo,Dai Tabuchi,Geor

## Get only 3 actor's name

In [6]:
#Fill all NaN with empty string
movie_df.fillna('', inplace=True)

In [7]:
movie_df['cast_name'] = movie_df['cast_name'].apply(lambda x: str(x.split(',')[:3])[1:-1] if len(x.split(',')) > 3 else str(x))

## Recommendation base on Weighted Rating

I use the TMDB Ratings to come up with our Top Movies Chart. I will use IMDB's weighted rating formula to construct my chart. Mathematically, it is represented as follows:

![](https://image.ibb.co/jYWZp9/wr.png)

v is the number of votes for the movie <br>
m is the minimum votes required to be listed in the chart<br>
R is the average rating of the movie<br>
C is the mean vote across the whole report<br>

In [8]:
C= movie_df['vote_average'].mean()

#minimum vote required, only 10% of the highest vote_count 
m= movie_df['vote_count'].quantile(0.9)

#get 10% of top vote_count movies
q_movies = movie_df.copy().loc[movie_df['vote_count'] >= m]
q_movies.shape

(481, 15)

In [9]:
#function for count weight rating
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
#insert weight rating into new columns ['score']
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#sort movies base on weigth rating
q_movies = q_movies.sort_values('score', ascending=False)

In [11]:
#get top 10 weigthed rating
q_movies.head(10)[['id','title','genres','production_companies','popularity','vote_average','vote_count','cast_name','producer','director','release_date',]]

,id,title,genres,production_companies,popularity,vote_average,vote_count,cast_name,producer,director,release_date
1881,278,The Shawshank Redemption,"Drama,Crime",Castle Rock Entertainment,136.747729,8.5,8205,"'Tim Robbins', 'Morgan Freeman', 'Bob Gunton'",Niki Marvin,Frank Darabont,1994-09-23
662,550,Fight Club,Drama,"Regency Enterprises,Fox 2000 Pictures,Taurus Film,Linson Films,Atman Entertainment,Knickerbocker Films",146.757391,8.3,9413,"'Edward Norton', 'Brad Pitt', 'Meat Loaf'","Art Linson,Ross Grayson Bell",David Fincher,1999-10-15
65,155,The Dark Knight,"Drama,Action,Crime,Thriller","DC Comics,Legendary Pictures,Warner Bros.,DC Entertainment,Syncopy",187.322927,8.2,12002,"'Christian Bale', 'Heath Ledger', 'Aaron Eckhart'","Charles Roven,Christopher Nolan,Emma Thomas,Lorne Orleans",Christopher Nolan,2008-07-16
3232,680,Pulp Fiction,"Thriller,Crime","Miramax Films,A Band Apart,Jersey Films",121.463076,8.3,8428,"'John Travolta', 'Samuel L. Jackson', 'Uma Thurman'",Lawrence Bender,Quentin Tarantino,1994-10-08
96,27205,Inception,"Action,Thriller,Science Fiction,Mystery,Adventure","Legendary Pictures,Warner Bros.,Syncopy",167.583710,8.1,13752,"'Leonardo DiCaprio', 'Ellen Page', 'Tom Hardy'","Christopher Nolan,Emma Thomas",Christopher Nolan,2010-07-14
3337,238,The Godfather,"Drama,Crime","Paramount Pictures,Alfran Productions",143.659698,8.4,5893,"'Marlon Brando', 'Al Pacino', 'James Caan'",Albert S. Ruddy,Francis Ford Coppola,1972-03-14
95,157336,Interstellar,"Adventure,Drama,Science Fiction","Paramount Pictures,Legendary Pictures,Warner Bros.,Syncopy,Lynda Obst Productions",724.247784,8.1,10867,"'Matthew McConaughey', 'Jessica Chastain', 'Anne Hathaway'","Christopher Nolan,Emma Thomas,Lynda Obst",Christopher Nolan,2014-11-05
809,13,Forrest Gump,"Comedy,Drama,Romance",Paramount Pictures,138.133331,8.2,7927,"'Tom Hanks', 'Robin Wright', 'Gary Sinise'","Wendy Finerman,Steve Tisch,Steve Starkey",Robert Zemeckis,1994-07-06
329,122,The Lord of the Rings: The Return of the King,"Adventure,Fantasy,Action","WingNut Films,New Line Cinema",123.630332,8.1,8064,"'Elijah Wood', 'Ian McKellen', 'Viggo Mortensen'","Barrie M. Osborne,Peter Jackson",Peter Jackson,2003-12-01
1990,1891,The Empire Strikes Back,"Adventure,Action,Science Fiction","Lucasfilm,Twentieth Century Fox Film Corporation",78.517830,8.2,5879,"'Mark Hamill', 'Harrison Ford', 'Carrie Fisher'","Gary Kurtz,Howard G. Kazanjian",Irvin Kershner,1980-05-17


## Recomendation base on Overview

In [12]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

def create_cosine_sim(df, col):
    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer(stop_words='english')

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(df[col])
    
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    return cosine_sim

In [13]:
def get_recommendations(df, title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie (pair of the index and the similarity scores)
    # by taking 1 row that represent the movie in cosine_sim matrix
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the similarity scores, so we can get the most similar movie with the title that was given
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies, exclude index 0 because it is the title that was given
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [14]:
cosine_sim = create_cosine_sim(movie_df, 'overview')
cosine_sim.shape

(4803, 4803)

In [15]:
#Construct a reverse map of indices and movie titles, so we can get the index base on movie's title
indices = pd.Series(movie_df.index, index=movie_df['title']).drop_duplicates()

In [16]:
get_recommendations(movie_df, 'The Dark Knight Rises', cosine_sim)

65      The Dark Knight                        
299     Batman Forever                         
428     Batman Returns                         
1359    Batman                                 
3854    Batman: The Dark Knight Returns, Part 2
119     Batman Begins                          
2507    Slow Burn                              
1181    JFK                                    
210     Batman & Robin                         
9       Batman v Superman: Dawn of Justice     
Name: title, dtype: object

In [17]:
get_recommendations(movie_df, 'The Lord of the Rings: The Return of the King', cosine_sim)

330     The Lord of the Rings: The Two Towers            
262     The Lord of the Rings: The Fellowship of the Ring
111     Transformers                                     
22      The Hobbit: The Desolation of Smaug              
3733    Night Watch                                      
3622    Once Upon a Time in the West                     
4400    Sparkler                                         
50      Prince of Persia: The Sands of Time              
2027    I Am Sam                                         
3560    Ouija                                            
Name: title, dtype: object

In [18]:
get_recommendations(movie_df, 'Fight Club', cosine_sim)

3619    UHF                  
2828    Project X            
2585    The Hurt Locker      
2344    Raging Bull          
2023    The Animal           
1414    Blast from the Past  
4044    Go for It!           
3515    Freaky Deaky         
4045    Dancer, Texas Pop. 81
4760    This Is Martin Bonner
Name: title, dtype: object

## Recomendation base on Overview

In [19]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [20]:
#function for make set of unique values of a columns
def make_set(df,col):
    set_keywords = set()
    for keyword in df[col].str.split(',').values:
        if isinstance(keyword, float): continue
        set_keywords = set_keywords.union(keyword)
    return set_keywords

#function for make a dictionary from a columns
def count_word(df, col):
    list_word = make_set(df,col)
    #make dict for count words
    keyword_count = dict()
    
    #fill dict key with words set default value to 0
    for i in list_word: keyword_count[i] = 0
    
    #loop all data from column, split all the word, set it as a list
    for list_key in df[col].str.split(','):  
        #ignore NaN
        if type(list_key) == float and pd.isnull(list_key): continue  
        #loop all tha data, remove word that's not in our list_word
        for key in [s for s in list_key if s in list_word]:
            #add 1 count for every word that found
            keyword_count[key] += 1
            
    #return unsorted dict
    return keyword_count

In [21]:
#check keyword that only appears once, twice, three times, and four times
sum1 = sum2 = sum3 = sum4 = 0
for i in count_word(movie_df,'keywords').values():
    if i == 1 : sum1 += 1
    elif i == 2 : sum2 += 1
    elif i == 3 : sum3 += 1
    else: sum4 += 1
print(sum1 , sum2 , sum3, sum4)

5273 1547 804 2145


In [22]:
#since there are 5273 keywords that appears once and 1547 that appears twice
#to reduce the size of matrix, we will only use key words that appears three times or more
keywords_list = []
for k,v in zip(count_word(movie_df,'keywords').keys(),count_word(movie_df,'keywords').values()):
    if v > 2: keywords_list.append(k)

In [23]:
#function to reduce keywords and remove space for keywords that have 2 or more words and replace , with space
def filter_keywords(param):
    words = []
    for i in param.split(','):
#         print(i)
        if i in keywords_list:
            stem_i = stemmer.stem(i)
            words.append(i)
    return re.sub(r' |\'|\[|\]','',str(words)).lower().replace(',',' ')

In [24]:
movie_df['keywords'] = movie_df['keywords'].apply(filter_keywords)

In [25]:
cosine_sim = create_cosine_sim(movie_df, 'overview')
cosine_sim.shape

(4803, 4803)

In [26]:
get_recommendations(movie_df, 'The Dark Knight Rises', cosine_sim)

65      The Dark Knight                        
299     Batman Forever                         
428     Batman Returns                         
1359    Batman                                 
3854    Batman: The Dark Knight Returns, Part 2
119     Batman Begins                          
2507    Slow Burn                              
1181    JFK                                    
210     Batman & Robin                         
9       Batman v Superman: Dawn of Justice     
Name: title, dtype: object

In [27]:
get_recommendations(movie_df, 'The Lord of the Rings: The Return of the King', cosine_sim)

330     The Lord of the Rings: The Two Towers            
262     The Lord of the Rings: The Fellowship of the Ring
111     Transformers                                     
22      The Hobbit: The Desolation of Smaug              
3733    Night Watch                                      
3622    Once Upon a Time in the West                     
4400    Sparkler                                         
50      Prince of Persia: The Sands of Time              
2027    I Am Sam                                         
3560    Ouija                                            
Name: title, dtype: object

In [28]:
get_recommendations(movie_df, 'Fight Club', cosine_sim)

3619    UHF                  
2828    Project X            
2585    The Hurt Locker      
2344    Raging Bull          
2023    The Animal           
1414    Blast from the Past  
4044    Go for It!           
3515    Freaky Deaky         
4045    Dancer, Texas Pop. 81
4760    This Is Martin Bonner
Name: title, dtype: object

## Recommendation Keywords, Genres, Director, Cast

In [29]:
movie_df['director'] = movie_df['director'].apply(lambda x: x.replace(',',' ').lower())
movie_df['cast_name'] = movie_df['cast_name'].apply(lambda x: re.sub(r' |\'|\[|\]','',x.lower()).replace(',',' '))
movie_df['genres'] = movie_df['genres'].apply(lambda x: x.replace(',',' ').lower())

In [30]:
movie_df['soup'] = movie_df[['keywords','cast_name','director','genres']].apply(lambda x: ''.join(x['keywords']) + ' '+ ''.join(x['cast_name'])+ ' ' + ''.join(x['director']) +' '+ ''.join(x['genres']), axis=1)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(movie_df['soup'])

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [33]:
get_recommendations(movie_df, 'The Dark Knight Rises', cosine_sim2)

65      The Dark Knight         
119     Batman Begins           
4638    Amidst the Devil's Wings
4099    Harsh Times             
1503    Takers                  
3073    Romeo Is Bleeding       
1253    Kiss of Death           
2154    Street Kings            
2502    Zulu                    
2915    Trash                   
Name: title, dtype: object

In [34]:
get_recommendations(movie_df, 'The Lord of the Rings: The Return of the King', cosine_sim2)

330     The Lord of the Rings: The Two Towers            
262     The Lord of the Rings: The Fellowship of the Ring
19      The Hobbit: The Battle of the Five Armies        
98      The Hobbit: An Unexpected Journey                
22      The Hobbit: The Desolation of Smaug              
107     Warcraft                                         
379     Conan the Barbarian                              
208     The 13th Warrior                                 
2390    Red Sonja                                        
1686    The Borrowers                                    
Name: title, dtype: object

In [35]:
get_recommendations(movie_df, 'Fight Club', cosine_sim2)

1553    Se7en                              
1161    The Social Network                 
946     The Game                           
100     The Curious Case of Benjamin Button
1010    Panic Room                         
693     Gone Girl                          
3283    Down in the Valley                 
354     The Girl with the Dragon Tattoo    
1512    A History of Violence              
421     Zodiac                             
Name: title, dtype: object